All time listening data ever

In [18]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly.express as px

spotifyA = pd.read_json('testing\Streaming_History_Audio_2018-2024_0.json')
spotifyB = pd.read_json('testing\Streaming_History_Audio_2024-2025_1.json')
spotify = pd.concat([spotifyA, spotifyB], ignore_index=True)
youtube = pd.read_json('watch-history.json')

In [19]:
spotify.rename(columns={'ts':'endTime','master_metadata_track_name':'title','master_metadata_album_artist_name':'artist', 'master_metadata_album_album_name':'album'}, inplace=True)
spotify['endTime'] = pd.to_datetime(spotify['endTime']) 
spotify['date'] = pd.to_datetime(spotify['endTime'].dt.date)
spotify['time'] = spotify['endTime'].dt.strftime('%H:%M')
spotify['hour'] = spotify['endTime'].dt.strftime('%H') #need hour for graph
spotify['hour'] = pd.to_numeric(spotify['hour']) 
spotify['year'] = spotify['endTime'].dt.strftime('%Y')
spotify['month'] = spotify['endTime'].dt.strftime('%m')
spotify['month'] = pd.to_numeric(spotify['month'])

spotify.head()

,endTime,platform,ms_played,conn_country,ip_addr,title,artist,album,spotify_track_uri,episode_name,...,shuffle,skipped,offline,offline_timestamp,incognito_mode,date,time,hour,year,month
0,2018-06-27 21:05:02+00:00,"Android OS 5.1.1 API 22 (LGE, Nexus 4)",221176,US,173.90.210.72,No Lie,Sean Paul,No Lie,spotify:track:1Vb4HQnN2kZ5Y2KgYF5TDV,None,...,True,False,False,NaN,False,2018-06-27,21:05,21,2018,6
1,2018-06-27 21:07:39+00:00,"Android OS 5.1.1 API 22 (LGE, Nexus 4)",156481,US,173.90.210.72,I'd Rather Go Blind - Live,Dua Lipa,Live Acoustic EP,spotify:track:5nMkd5cq4JPZqAoQTXv8Wm,None,...,True,False,False,NaN,False,2018-06-27,21:07,21,2018,6
2,2018-06-27 21:08:13+00:00,"Android OS 5.1.1 API 22 (LGE, Nexus 4)",16041,US,173.90.210.72,Room for 2,Dua Lipa,Room For 2,spotify:track:4U1TYTYFoFXfhTEGwLlTH1,None,...,True,False,False,NaN,False,2018-06-27,21:08,21,2018,6
3,2018-06-27 21:09:34+00:00,"Android OS 5.1.1 API 22 (LGE, Nexus 4)",62662,US,173.90.210.72,Savior,Iggy Azalea,Savior,spotify:track:1rMo0Fuv4vYRGT78bZQohI,None,...,True,False,False,NaN,False,2018-06-27,21:09,21,2018,6
4,2018-06-27 21:22:09+00:00,"Android OS 5.1.1 API 22 (LGE, Nexus 4)",139807,US,173.90.210.72,"Let's Walk Across This Forest, I Can Feel Ever...",In Love With a Ghost,Healing,spotify:track:2XizvxDAomTbA3KOnOgGcc,None,...,True,False,False,NaN,False,2018-06-27,21:22,21,2018,6


In [20]:
youtube = youtube[youtube['header']=='YouTube Music'] #only take data from youtube music 
youtube = youtube.drop(['titleUrl', 'products', 'activityControls', 'description', 'details', 'header'], axis=1)

#Convert youtube ListTime to datetime
youtube['ListTime'] = pd.to_datetime(youtube['time'], errors='coerce', utc=True)
youtube['date'] = pd.to_datetime(youtube['ListTime'].dt.date) 
youtube['time'] = youtube['ListTime'].dt.strftime('%H:%M')
youtube['hour'] = youtube['ListTime'].dt.strftime('%H') #need hour for graph
youtube['hour'] = pd.to_numeric(youtube['hour'])
youtube['year'] = youtube['ListTime'].dt.strftime('%Y')
youtube['month'] = youtube['ListTime'].dt.strftime('%m')
youtube['month'] = pd.to_numeric(youtube['month'])

#Eliminate 'Watched' from song titles
def delete_watched(value): 
    pattern = r'^Watched '
    if re.match(pattern, value):
        cleaned = re.sub(r'^Watched ', '', value)
        return cleaned

youtube['title'] = youtube['title'].apply(delete_watched) 

#Get artist names from list in subtitles
youtube = youtube[~youtube['subtitles'].apply(lambda x: isinstance(x, float))] #drop float rows

def get_name(value): #Index artist names 
    return value[0]['name']

youtube['artist'] = youtube['subtitles'].apply(get_name) #Make artist column 

def delete_topic(value): #delete '- Topic' in names 
    pattern = r'.*\- Topic$'
    if re.match(pattern, value):
        cleaned = re.sub(r' \- Topic', '', value)
        return cleaned, True #add cleaned_flag for songs with author
    else: return value, False

youtube[['artist', 'cleaned_flag']] = youtube['artist'].apply(delete_topic).apply(pd.Series)

#Eliminate random characters from titles and artists
youtube['title'] = youtube['title'].str.replace(r"[•·]", " ", regex=True)
youtube['artist'] = youtube['artist'].str.replace(r"[•·]", " ", regex=True)

artists = youtube[youtube['cleaned_flag']==True]['artist'].unique() #Create list of clean artists

sorted_artists = sorted(artists, key=len, reverse=True) #sort greatest to shortest to match longest

#Find artist from artist name in title
def find_artist(title): #Function to find artist name in title
    for artist in artists:
        if artist.lower() in title.lower():
            return artist
    return None
for idx, row in youtube[~youtube['cleaned_flag']].iterrows():
    artist_found = find_artist(row['title']) #change artist if found in title
    if artist_found:
        youtube.at[idx, 'artist'] = artist_found 

#Clean random words from song title
def clean_title(title):
    for artist in artists: #eliminate artist from title
        pattern = re.compile(re.escape(artist), re.IGNORECASE)
        title = pattern.sub('', title)
    cuts = ['(Un-Official Video)', '(Official Video)', '(lyrics)', '-', '(Official)', '(Audio)', '(Official Music Video)', '(feat. )']
    for phrase in cuts: #eliminate phrases from title
        pattern = re.compile(re.escape(phrase), re.IGNORECASE)
        title = pattern.sub('', title)
    title = title.strip()
    return title
youtube['title'] = youtube['title'].apply(clean_title)

In [24]:
spotify2 = spotify[['title', 'artist', 'date', 'hour', 'month', 'year']].copy()
youtube2 = youtube[['title', 'artist', 'date', 'hour', 'month', 'year']].copy()
 

music = pd.concat([spotify2, youtube2], ignore_index=True)

In [29]:


song_counts = music.groupby(['title', 'artist']).size().reset_index(name='count')
top10_songs = song_counts.sort_values('count', ascending=False)

print("top songs")
top10_songs.head(10) #print top songs with artist 

top songs


,title,artist,count
4698,The Gambler,fun.,96
1799,Get Used to It,Ricky Montgomery,93
5152,Victoria,Jukebox The Ghost,90
4102,She Wants Me (To Be Loved),The Happy Fits,89
4872,This December,Ricky Montgomery,87
4020,Scrawny,Wallows,85
2654,Last Night,Ricky Montgomery,85
4566,Talk to You,Ricky Montgomery,82
4053,Settle Down,Ricky Montgomery,81
1628,Favorite Liar,The Wrecks,80


In [30]:
artist_counts = music.groupby('artist').size().reset_index(name='count')
top10_artists = artist_counts.sort_values('count', ascending=False)

print("top artists")
top10_artists.head(10) #print top songs with artist 

top artists


,artist,count
1283,Twenty One Pilots,2409
18,AJR,1819
528,Imagine Dragons,1534
912,Panic! At The Disco,1530
1200,The Happy Fits,1281
994,Ricky Montgomery,1199
614,Jukebox The Ghost,1170
1423,fun.,1119
1432,half•alive,960
1071,Shawn Mendes,764


In [28]:
year_counts = music.groupby('year').size().reset_index(name='count')
big_years = year_counts.sort_values('count', ascending=False).head(10)

big_years.head() #print top songs with artist 

,year,count
6,2024,6399
3,2021,5182
5,2023,5164
4,2022,5002
2,2020,3490


In [ ]:
album_counts = music.groupby('album').size().reset_index(name='count')
top10_albums = album_counts.sort_values('count', ascending=False)

top10_albums.head(10) #print top songs with artist 

,album,count
223,Blurryface,481
1712,Vessel,445
1677,Twenty One Pilots,402
1666,Trench,344
1552,The Click,328
